# Intersection over Unity score

In [ ]:
import csv
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd

import numpy as np
import seaborn as sns
import datetime as datetime
from netCDF4 import num2date, date2num
import zooniverse as zn

from PIL import Image
import matplotlib.patches as patches
import matplotlib.image as mpimage

import numba as nb
from collections import defaultdict
import itertools

%load_ext autoreload
%autoreload 2

In [ ]:
classfile_in = '../zooniverse_raw/sugar-flower-fish-or-gravel-classifications_18_12_16.csv'
subject_in = '../zooniverse_raw/sugar-flower-fish-or-gravel-subjects_18_11_05.csv'

In [ ]:
classification_data = zn.parse_classifications(classfile_in,json_columns=['metadata', 'annotations', 'subject_data']);
prac_dataset = classification_data[np.logical_and(classification_data.workflow_id == 8072,
                                                  classification_data.workflow_version == 24.13) ]; prac_dataset.head()

In [ ]:
unique_users = list(prac_dataset['user_name'].unique())

In [ ]:
# Restrict comparision only to people who did more than x classifications
unique_users = []
for u,user in prac_dataset.groupby('user_name'):
    if len(user) > 20:
        unique_users.append(u)

In [ ]:
image_width =2100
image_height=1400
annos_prac = defaultdict(list)
for i, r in prac_dataset.iterrows():
    if len(r['annotations']['value']) > 0 :
        for a in r['annotations']['value']:
            x = np.max([0., a['x']]); y = np.max([0., a['y']])
            w = np.min([image_width - a['x'], a['width']])
            h = np.min([image_height - a['y'], a['height']])
            annos_prac[r['subject_ids']].append([
                r['user_name'],
                [x, y, w, h],
                a['tool_label']
            ])
    else:   # No annotation
        annos_prac[r['subject_ids']].append([
            r['user_name'],
            [None, None, None, None],
            None
        ])

In [ ]:
subjects_prac = list(annos_prac.keys()); len(subjects_prac)

In [ ]:
annos_prac.keys()

In [ ]:
def get_comp_data(subject_id, user1, user2, annos):
    subj_annos = annos[subject_id]
    users = list(set([a[0] for a in subj_annos]))
    if user1 in users and user2 in users:
        annos1 = [a[1:] for a in subj_annos if a[0] == user1]
        annos2 = [a[1:] for a in subj_annos if a[0] == user2]
        if annos1[0][1] is None and annos2[0][1] is None:
            iou = 1.
        elif annos1[0][1] is None:
            iou = 0.
        elif annos2[0][1] is None:
            iou = 0.
        else:
            iou = iou_multi_class(annos1, annos2)
        return iou
    else:
        return None

In [ ]:
def iou_multi_class(annos1, annos2):
    """For two users"""
    classes = ['Sugar', 'Flower', 'Fish', 'Gravel']
    i_tot = 0; u_tot = 0
    for c in classes:
        ac1 = [a[0] for a in annos1 if a[1] == c]
        ac2 = [a[0] for a in annos2 if a[1] == c]
        if len(ac1) == 0 and len(ac2) == 0:   # None of the users has this class
            i = 0; u = 0
        elif len(ac1) == 0 or len(ac2) == 0:
            i = 0
            u = compute_area_sum(ac1 + ac2)
        else:
            i, u = iou_one_class(ac1, ac2)
        i_tot += i; u_tot += u
    return i_tot / u_tot

In [ ]:
def iou_one_class(annos1, annos2):
    """annos1 and annos2 are lists with the coordinates of each annotation"""
    import itertools
    # All permutations
    i = 0
    for a1 in annos1:
        for a2 in annos2:
            i += compute_intersect(a1, a2)
    # Need to compute the overlap between the boxes of 1 and 2
    i1 = 0; i2 = 0
    for b1, b2 in itertools.combinations(annos1, 2):
        i1 += compute_intersect(b1, b2)
    for b1, b2 in itertools.combinations(annos2, 2):
        i2 += compute_intersect(b1, b2)
    u = compute_area_sum(annos1 + annos2) - i# - i1 -i2
    
    iou = i/u  #(i - i1 - i2) / u
    return i, u

In [ ]:
def compute_intersect(a1, a2):
    """https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/"""
    x1a, y1a, x2a, y2a = wh2xy(*a1)
    x1b, y1b, x2b, y2b = wh2xy(*a2)
    xA = max(x1a, x1b)
    yA = max(y1a, y1b)
    xB = min(x2a, x2b)
    yB = min(y2a, y2b)
    i = max(0, xB - xA) * max(0, yB - yA) # max(0, xB - xA + 1) * max(0, yB - yA + 1)
    return i

In [ ]:
def compute_area_sum(annos):
    areas = [a[2] * a[3] for a in annos]
    return np.sum(areas)

In [ ]:
def wh2xy(x, y, w, h):
    return x, y, x+w, y+h

In [ ]:
agreement  = defaultdict(list)
for s in subjects_prac:
    for u in unique_users:
        agreement[u].append(get_comp_data(s, 'sbony', u, annos_prac))

In [ ]:
df = pd.DataFrame(data = {'agreement': [np.mean([a for a in agreement[u] if a is not None]) for u in unique_users]}, index=unique_users)

In [ ]:
df.sort_values('agreement', axis=0, ascending=False, inplace=True)

In [ ]:
fig = plt.figure(figsize=(5, 15))
sns.barplot(x=df['agreement'], y=df.index,color="grey")
sns.despine(offset=10,left=True)
fig.set_dpi(300)